### Wordnet Synsets for OES categories using O-net grouping

In [42]:
import numpy as np 
import pandas as pd
from nltk.corpus import wordnet


In [303]:
occu = pd.read_table("Occupations.tsv")
clusters = pd.read_table("CareerClusters.tsv")

In [304]:
occu.head()

Code                                        Occupation  \
0  17-3021.00  Aerospace Engineering and Operations Technicians   
1  17-2011.00                               Aerospace Engineers   
2  17-2021.00                            Agricultural Engineers   
3  17-1011.00          Architects, Except Landscape and Naval     
4  17-3011.00                  Architectural and Civil Drafters   

                     Job Family  
0  Architecture and Engineering  
1  Architecture and Engineering  
2  Architecture and Engineering  
3  Architecture and Engineering  
4  Architecture and Engineering

In [305]:
clusters.head()

Career Cluster        Career Pathway        Code  \
0  Agriculture, Food & Natural Resources  Agribusiness Systems  11-9013.03   
1  Agriculture, Food & Natural Resources  Agribusiness Systems  13-1021.00   
2  Agriculture, Food & Natural Resources  Agribusiness Systems  11-9013.02   
3  Agriculture, Food & Natural Resources  Agribusiness Systems  13-1074.00   
4  Agriculture, Food & Natural Resources  Agribusiness Systems  11-9013.00   

                                          Occupation  
0                              Aquacultural Managers  
1        Buyers and Purchasing Agents, Farm Products  
2                            Farm and Ranch Managers  
3                             Farm Labor Contractors  
4  Farmers, Ranchers, and Other Agricultural Mana...

In [306]:
onetM = pd.merge(occu, clusters, on=["Code"])

In [307]:
onetM['fullCode'] = onetM.Code
onetM['SCode'] = onetM.Code.apply(lambda x: x[7:])
onetM['Code'] = onetM.Code.apply(lambda x: x[:7])
onetM.head(42)


In [309]:
onetM['word_tokens'] = onetM.Occupation_y.apply(tokenizer.tokenize)

In [310]:
oes = pd.read_table("oesCat.tsv")
oes.head()

Code                            Title
0  11-0000           Management Occupations
1  11-1000                   Top Executives
2  11-1010                 Chief Executives
3  11-1011                 Chief Executives
4  11-1020  General and Operations Managers

In [311]:
onet = pd.merge(onetM, oes, on="Code")

In [312]:
onet.head()

## Pull Summary Reports for roles from O*net.org

In [317]:
baseurl = "https://www.onetonline.org/link/summary/"
#from urllib.request import Request, urlopen, urlretrieve
#from urllib.error import HTTPError
#from urllib.error import HTTPError
#from urllib.error import HTTPError
from bs4 import BeautifulSoup
from downloader import Downloader

In [318]:
D = Downloader()

In [356]:
def getPages(col):
    pages = []
    for i in col:
        html = D(baseurl+i)
        pages.append(html['html'])
    return pages

In [358]:
onetReports = getPages(onet.fullCode)

Downloading:  https://www.onetonline.org/link/summary/17-3021.00
Downloading:  https://www.onetonline.org/link/summary/17-2011.00
Downloading:  https://www.onetonline.org/link/summary/17-2021.00
Downloading:  https://www.onetonline.org/link/summary/17-1011.00
Downloading:  https://www.onetonline.org/link/summary/17-3011.00
Downloading:  https://www.onetonline.org/link/summary/17-3011.01
Downloading:  https://www.onetonline.org/link/summary/17-3011.02
Downloading:  https://www.onetonline.org/link/summary/17-3027.01
Downloading:  https://www.onetonline.org/link/summary/17-3027.00
Downloading:  https://www.onetonline.org/link/summary/17-2141.02
Downloading:  https://www.onetonline.org/link/summary/17-2141.01
Downloading:  https://www.onetonline.org/link/summary/17-2141.00
Downloading:  https://www.onetonline.org/link/summary/17-2199.01
Downloading:  https://www.onetonline.org/link/summary/17-2199.03
Downloading:  https://www.onetonline.org/link/summary/17-2199.00
Downloading:  https://www

In [361]:
onetReports.to_csv("onetReports.txt", sep="\t")

In [364]:
def roleRep(col):
    summaries = []
    for i in col:
        soup = BeautifulSoup(i)
        summaries.append(soup.find("p").get_text())
    return summaries

In [365]:
onet['summary'] = roleRep(onetReports)

/usr/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [404]:
def roleSampls(col):
    sampls = []
    for i in col:
        soup = BeautifulSoup(i)
        sample_Titles = soup.find("p").next_sibling.get_text()
        for i,char in enumerate(sample_Titles):
            if char == ":":
                pos = i+2
                break
        sampls.append(sample_Titles[pos:-1])
    return sampls


In [405]:
onet['sampleTitles'] = roleSampls(onetReports)

/usr/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [406]:
onet.head()

Code                                      Occupation_x  \
0  17-3021  Aerospace Engineering and Operations Technicians   
1  17-2011                               Aerospace Engineers   
2  17-2021                            Agricultural Engineers   
3  17-1011          Architects, Except Landscape and Naval     
4  17-3011                  Architectural and Civil Drafters   

                     Job Family  \
0  Architecture and Engineering   
1  Architecture and Engineering   
2  Architecture and Engineering   
3  Architecture and Engineering   
4  Architecture and Engineering   

                                   Career Cluster  \
0                                   Manufacturing   
1  Science, Technology, Engineering & Mathematics   
2           Agriculture, Food & Natural Resources   
3                     Architecture & Construction   
4                     Architecture & Construction   

                                 Career Pathway  \
0  Manufacturing Production Process Development   
1                    Engineering and Technology   
2         Power, Structural & Technical Systems   
3                       Design/Pre-Construction   
4                       Design/Pre-Construction   

                                       Occupation_y    fullCode SCode  \
0  Aerospace Engineering and Operations Technicians  17-3021.00   .00   
1                               Aerospace Engineers  17-2011.00   .00   
2                            Agricultural Engineers  17-2021.00   .00   
3            Architects, Except Landscape and Naval  17-1011.00   .00   
4                  Architectural and Civil Drafters  17-3011.00   .00   

                                         word_tokens  \
0  [Aerospace, Engineering, and, Operations, Tech...   
1                             [Aerospace, Engineers]   
2                          [Agricultural, Engineers]   
3        [Architects, Except, Landscape, and, Naval]   
4              [Architectural, and, Civil, Drafters]   

                                              Title  \
0  Aerospace Engineering and Operations Technicians   
1                               Aerospace Engineers   
2                            Agricultural Engineers   
3            Architects, Except Landscape and Naval   
4                  Architectural and Civil Drafters   

                                             summary  \
0  Operate, install, calibrate, and maintain inte...   
1  Perform engineering duties in designing, const...   
2  Apply knowledge of engineering technology and ...   
3  Plan and design structures, such as private re...   
4  Prepare detailed drawings of architectural and...   

                                        sampleTitles  \
0  Avionics Technician, Avionics Test Technician,...   
1  Aeronautical Engineer, Aerospace Engineer, Aer...   
2  Agricultural Engineer, Agricultural Safety and...   
3  Architect; Architectural Project Manager; Desi...   
4  f more specific occupations. For additional in...   

                                      summary_tokens  \
0  [Operate, install, calibrate, and, maintain, i...   
1  [Perform, engineering, duties, in, designing, ...   
2  [Apply, knowledge, of, engineering, technology...   
3  [Plan, and, design, structures, such, as, priv...   
4  [Prepare, detailed, drawings, of, architectura...   

                                      sampleT_tokens  
0  [Avionics, Technician, Avionics, Test, Technic...  
1  [Aeronautical, Engineer, Aerospace, Engineer, ...  
2  [Agricultural, Engineer, Agricultural, Safety,...  
3  [Architect, Architectural, Project, Manager, D...  
4  [f, more, specific, occupations, For, addition...

In [407]:
#testing out onet pages parser vvv
#html = D("https://www.onetonline.org/link/summary/13-2011.01")


In [408]:
'''
#print(html['html'])
soup = BeautifulSoup(html['html'])

Summary = soup.find("p").get_text()
sample_Titles = soup.find("p").next_sibling.get_text()
for i,char in enumerate(sample_Titles):
    if char == ":":
        pos = i+2
        break
sample_Titles = sample_Titles[pos:]
print(sample_Titles)
#sample_Titles = [sib for sib in soup.find("p").next_siblings]
#print(sample_Titles[0])
'''

'\n#print(html[\'html\'])\nsoup = BeautifulSoup(html[\'html\'])\n\nSummary = soup.find("p").get_text()\nsample_Titles = soup.find("p").next_sibling.get_text()\nfor i,char in enumerate(sample_Titles):\n    if char == ":":\n        pos = i+2\n        break\nsample_Titles = sample_Titles[pos:]\nprint(sample_Titles)\n#sample_Titles = [sib for sib in soup.find("p").next_siblings]\n#print(sample_Titles[0])\n'

In [409]:
titles = onet.filter(['Occupation_x', 'Occupation_y', 'summary', 'sampleTitles'], axis=1)
#titles = pd.merge(titles, pd.DataFrame(onetM.Occupation_y), right_index=True, left_index=True)
titles
titles.to_csv("Titles.txt", sep="\t")


In [411]:
onet['summary_tokens'] = onet.summary.apply(tokenizer.tokenize)
onet['sampleT_tokens'] = onet.sampleTitles.apply(tokenizer.tokenize)

In [412]:
onet.head()

Code                                      Occupation_x  \
0  17-3021  Aerospace Engineering and Operations Technicians   
1  17-2011                               Aerospace Engineers   
2  17-2021                            Agricultural Engineers   
3  17-1011          Architects, Except Landscape and Naval     
4  17-3011                  Architectural and Civil Drafters   

                     Job Family  \
0  Architecture and Engineering   
1  Architecture and Engineering   
2  Architecture and Engineering   
3  Architecture and Engineering   
4  Architecture and Engineering   

                                   Career Cluster  \
0                                   Manufacturing   
1  Science, Technology, Engineering & Mathematics   
2           Agriculture, Food & Natural Resources   
3                     Architecture & Construction   
4                     Architecture & Construction   

                                 Career Pathway  \
0  Manufacturing Production Process Development   
1                    Engineering and Technology   
2         Power, Structural & Technical Systems   
3                       Design/Pre-Construction   
4                       Design/Pre-Construction   

                                       Occupation_y    fullCode SCode  \
0  Aerospace Engineering and Operations Technicians  17-3021.00   .00   
1                               Aerospace Engineers  17-2011.00   .00   
2                            Agricultural Engineers  17-2021.00   .00   
3            Architects, Except Landscape and Naval  17-1011.00   .00   
4                  Architectural and Civil Drafters  17-3011.00   .00   

                                         word_tokens  \
0  [Aerospace, Engineering, and, Operations, Tech...   
1                             [Aerospace, Engineers]   
2                          [Agricultural, Engineers]   
3        [Architects, Except, Landscape, and, Naval]   
4              [Architectural, and, Civil, Drafters]   

                                              Title  \
0  Aerospace Engineering and Operations Technicians   
1                               Aerospace Engineers   
2                            Agricultural Engineers   
3            Architects, Except Landscape and Naval   
4                  Architectural and Civil Drafters   

                                             summary  \
0  Operate, install, calibrate, and maintain inte...   
1  Perform engineering duties in designing, const...   
2  Apply knowledge of engineering technology and ...   
3  Plan and design structures, such as private re...   
4  Prepare detailed drawings of architectural and...   

                                        sampleTitles  \
0  Avionics Technician, Avionics Test Technician,...   
1  Aeronautical Engineer, Aerospace Engineer, Aer...   
2  Agricultural Engineer, Agricultural Safety and...   
3  Architect; Architectural Project Manager; Desi...   
4  f more specific occupations. For additional in...   

                                      summary_tokens  \
0  [Operate, install, calibrate, and, maintain, i...   
1  [Perform, engineering, duties, in, designing, ...   
2  [Apply, knowledge, of, engineering, technology...   
3  [Plan, and, design, structures, such, as, priv...   
4  [Prepare, detailed, drawings, of, architectura...   

                                      sampleT_tokens  
0  [Avionics, Technician, Avionics, Test, Technic...  
1  [Aeronautical, Engineer, Aerospace, Engineer, ...  
2  [Agricultural, Engineer, Agricultural, Safety,...  
3  [Architect, Architectural, Project, Manager, D...  
4  [f, more, specific, occupations, For, addition...

In [413]:
#onet.to_csv("combined_onet.txt", sep="\t")



In [1]:
import numpy as np 
import pandas as pd
from nltk.corpus import wordnet

In [2]:
onet = pd.read_csv("data/combined_onet.txt", sep="\t")

### tokenization must be redone because those vals are read in as strings not converted to lists vvvv

In [3]:
from nltk.tokenize import RegexpTokenizer

In [4]:
tokenizer = RegexpTokenizer("[\w']+")
def WT(ls):
    col = []
    for i in ls:
        col.append(tokenizer.tokenize(i))
    return col


In [5]:
onet['word_tokens'] = onet.Occupation_y.apply(tokenizer.tokenize)


In [6]:
onet['summary_tokens'] = onet.summary.apply(tokenizer.tokenize)


In [7]:
onet['sampleT_tokens'] = onet.sampleTitles.apply(tokenizer.tokenize)


In [8]:
onet.tail()

Unnamed: 0     Code                                       Occupation_x  \
1082        1082  53-3041                        Taxi Drivers and Chauffeurs   
1083        1083  53-6041                                Traffic Technicians   
1084        1084  53-6061  Transportation Attendants, Except Flight Atten...   
1085        1085  53-6099                  Transportation Workers, All Other   
1086        1086  53-7073                                   Wellhead Pumpers   

                              Job Family  \
1082  Transportation and Material Moving   
1083  Transportation and Material Moving   
1084  Transportation and Material Moving   
1085  Transportation and Material Moving   
1086  Transportation and Material Moving   

                                Career Cluster  \
1082  Transportation, Distribution & Logistics   
1083  Transportation, Distribution & Logistics   
1084  Transportation, Distribution & Logistics   
1085  Transportation, Distribution & Logistics   
1086  Transportation, Distribution & Logistics   

                                         Career Pathway  \
1082                          Transportation Operations   
1083  Transportation Systems/Infrastructure Planning...   
1084                          Transportation Operations   
1085                          Transportation Operations   
1086                          Transportation Operations   

                                           Occupation_y    fullCode  SCode  \
1082                        Taxi Drivers and Chauffeurs  53-3041.00    0.0   
1083                                Traffic Technicians  53-6041.00    0.0   
1084  Transportation Attendants, Except Flight Atten...  53-6061.00    0.0   
1085                  Transportation Workers, All Other  53-6099.00    0.0   
1086                                   Wellhead Pumpers  53-7073.00    0.0   

                                            word_tokens  \
1082                   [Taxi, Drivers, and, Chauffeurs]   
1083                             [Traffic, Technicians]   
1084  [Transportation, Attendants, Except, Flight, A...   
1085              [Transportation, Workers, All, Other]   
1086                                [Wellhead, Pumpers]   

                                                  Title  \
1082                        Taxi Drivers and Chauffeurs   
1083                                Traffic Technicians   
1084  Transportation Attendants, Except Flight Atten...   
1085                  Transportation Workers, All Other   
1086                                   Wellhead Pumpers   

                                                summary  \
1082  Drive automobiles, vans, or limousines to tran...   
1083  Conduct field studies to determine traffic vol...   
1084  Provide services to ensure the safety and comf...   
1085  All transportation workers not listed separately.   
1086  Operate power pumps and auxiliary equipment to...   

                                           sampleTitles  \
1082  Cab Driver, Chauffeur, Driver, Limousine Drive...   
1083  Engineering Technician, Field Traffic Investig...   
1084  Bus Aide, Bus Assistant, Bus Attendant, Bus Mo...   
1085  cupations with a wide range of characteristics...   
1086  Gauger, Lease Operator, Multi-Skilled Operator...   

                                         summary_tokens  \
1082  [Drive, automobiles, vans, or, limousines, to,...   
1083  [Conduct, field, studies, to, determine, traff...   
1084  [Provide, services, to, ensure, the, safety, a...   
1085  [All, transportation, workers, not, listed, se...   
1086  [Operate, power, pumps, and, auxiliary, equipm...   

                                         sampleT_tokens  
1082  [Cab, Driver, Chauffeur, Driver, Limousine, Dr...  
1083  [Engineering, Technician, Field, Traffic, Inve...  
1084  [Bus, Aide, Bus, Assistant, Bus, Attendant, Bu...  
1085  [cupations, with, a, wide, range, of, characte...  
1086  [Gauger, Lease, Operator, Multi, Skilled, Oper...

### Make some ngrams using Onet data

In [9]:
from nltk.corpus import webtext
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.metrics import TrigramAssocMeasures
words = [w.lower() for w in webtext.words('Titles.txt')]
bcf = BigramCollocationFinder.from_words(words)
tcf = TrigramCollocationFinder.from_words(words)

In [10]:
bcf.nbest(BigramAssocMeasures.likelihood_ratio,4)

[('such', 'as'), ('.', 'may'), ('all', 'other'), ('operator', ',')]

In [11]:
tcf.nbest(TrigramAssocMeasures.likelihood_ratio,4)

[(',', 'such', 'as'),
 ('areas', 'such', 'as'),
 ('duties', 'such', 'as'),
 ('such', 'as', 'a')]

In [12]:
from nltk.corpus import stopwords
stopset = set(stopwords.words('english'))
filter_stops = lambda w: len(w) < 3 or w in stopset
bcf.apply_word_filter(filter_stops)
tcf.apply_word_filter(filter_stops)

In [13]:
bcf.nbest(BigramAssocMeasures.likelihood_ratio, 4)

[('listed', 'separately'),
 ('real', 'estate'),
 ('vice', 'president'),
 ('line', 'supervisors')]

In [14]:
tcf.nbest(TrigramAssocMeasures.likelihood_ratio, 4)

[('postsecondary', 'teach', 'courses'),
 ('teachers', 'primarily', 'engaged'),
 ('geographic', 'information', 'systems'),
 ('special', 'education', 'teacher')]

In [15]:
bcf.score_ngrams(BigramAssocMeasures.likelihood_ratio)[:5]

[(('listed', 'separately'), 1125.9969615711857),
 (('real', 'estate'), 577.3966406928769),
 (('vice', 'president'), 575.2095108203183),
 (('line', 'supervisors'), 536.9621261999703),
 (('machine', 'operator'), 498.56179999894584)]

In [16]:
tcf.score_ngrams(TrigramAssocMeasures.likelihood_ratio)[:5]


[(('postsecondary', 'teach', 'courses'), 1394.4536380112377),
 (('teachers', 'primarily', 'engaged'), 1366.999889309856),
 (('geographic', 'information', 'systems'), 1012.2440198719406),
 (('special', 'education', 'teacher'), 991.5522421408288),
 (('real', 'estate', 'appraiser'), 983.427370804294)]

### create synsets for titles

In [17]:
onet.columns

Index(['Unnamed: 0', 'Code', 'Occupation_x', 'Job Family', 'Career Cluster',
       'Career Pathway', 'Occupation_y', 'fullCode', 'SCode', 'word_tokens',
       'Title', 'summary', 'sampleTitles', 'summary_tokens', 'sampleT_tokens'],
      dtype='object')

## Role Text to title Matching


In [18]:
import nltk
from nltk.tokenize import sent_tokenize

In [19]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /home/iss/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('webtext')


[nltk_data] Downloading package stopwords to /home/iss/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/iss/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package webtext to /home/iss/nltk_data...
[nltk_data]   Unzipping corpora/webtext.zip.


True

In [45]:

from nltk.corpus import stopwords
english_stops = set(stopwords.words('english'))

In [46]:
onet['word_tokens_stopw'] = onet.word_tokens.apply(lambda x: [word for word in x if word not in english_stops])

In [47]:
onet['summary_tokens_stopw'] = onet.summary_tokens.apply(lambda x: [word for word in x if word not in english_stops])
onet['sample_tokens_stopw'] = onet.sampleT_tokens.apply(lambda x: [word for word in x if word not in english_stops])

In [48]:
def synMaker(ls):
    syns = []
    for i in ls:
        syns.append(wordnet.synsets(i))
    return syns
        

In [49]:
wordnet.synsets('operate')

[Synset('operate.v.01'),
 Synset('function.v.01'),
 Synset('operate.v.03'),
 Synset('manoeuver.v.03'),
 Synset('operate.v.05'),
 Synset('engage.v.10'),
 Synset('operate_on.v.01')]

In [50]:
#loops through words for job titles and retrieves all synsets for each
onet['syns'] = onet.word_tokens_stopw.apply(synMaker)
onet['summary_syns'] = onet.summary_tokens_stopw.apply(synMaker)
onet['sample_syns'] = onet.sample_tokens_stopw.apply(synMaker)

In [52]:
onet.syns[0]

[[Synset('aerospace.n.01')],
 [Synset('technology.n.01'),
  Synset('engineering.n.02'),
  Synset('engineering.n.03'),
  Synset('engineer.v.01'),
  Synset('mastermind.v.01')],
 [Synset('operations.n.01'),
  Synset('operation.n.01'),
  Synset('operation.n.02'),
  Synset('operation.n.03'),
  Synset('operation.n.04'),
  Synset('operation.n.05'),
  Synset('operation.n.06'),
  Synset('operation.n.07'),
  Synset('operation.n.08'),
  Synset('process.n.02'),
  Synset('mathematical_process.n.01'),
  Synset('operation.n.11')],
 [Synset('technician.n.01'), Synset('technician.n.02')]]

In [53]:
def lemming(ls):
    col = []
    for i in ls:
        newList = []
        for j in i:
            newList.append(j.lemmas())
        col.append(newList)
    return col


In [54]:
onet['lemmings'] = onet.syns.apply(lemming)
onet['summary_lem'] = onet.summary_syns.apply(lemming)
onet['sample_lem'] = onet.sample_syns.apply(lemming)

In [55]:
#onet.summarylem[4]

In [56]:
def synonyms(ls):
    col = []
    for i in ls:
        newList = []
        for j in i:
            #loops through the lists of synsets for each word getting lemmas or synonyms then
            #loops again through the lemmas returning just the names or words themselves
            #This looping syntax below is relatively new to me but I believe it loops through the in argument
            #and returns the values requested prior to the 'for' statment
            newList.append([lemma.name() for lemma in j.lemmas()])
        col.append(newList)
    return col

In [57]:
onet['synonyms'] = onet.syns.apply(synonyms)
onet['summary_synonyms'] = onet.summary_syns.apply(synonyms)
onet['sample_synonyms'] = onet.sample_syns.apply(synonyms)

In [58]:
#onet.summary_synonyms[4]

In [59]:
onet.syns[7]

[[Synset('automotive.a.01'), Synset('automotive.s.02')],
 [Synset('technology.n.01'),
  Synset('engineering.n.02'),
  Synset('engineering.n.03'),
  Synset('engineer.v.01'),
  Synset('mastermind.v.01')],
 [Synset('technician.n.01'), Synset('technician.n.02')]]

### Combine above functions into one to return a flat list of synonyms (does not seperate synonyms for multiple words)

In [32]:
### chained synonym getter passing in synsets returning flattened synonym list
def flatSynonym(synList):
    synonyms = []
    for synset in synList:
        for word in synset:
            for lemma in word.lemmas():
                synonyms.append(lemma.name())
    return synonyms
        
        

In [33]:
onet['flatSynonyms'] = onet.syns.apply(flatSynonym)
onet['summary_flatSynonyms'] = onet.summary_syns.apply(flatSynonym)
onet['sample_flatSynonyms'] = onet.sample_syns.apply(flatSynonym)

In [34]:
onet.summary_flatSynonyms[4][:5]

['fix', 'prepare', 'set_up', 'ready', 'gear_up']

In [35]:
onet.iloc[4]

Unnamed: 0                                                              4
Code                                                              17-3011
Occupation_x                             Architectural and Civil Drafters
Job Family                                   Architecture and Engineering
Career Cluster                                Architecture & Construction
Career Pathway                                    Design/Pre-Construction
Occupation_y                             Architectural and Civil Drafters
fullCode                                                       17-3011.00
SCode                                                                   0
word_tokens                         [Architectural, and, Civil, Drafters]
Title                                    Architectural and Civil Drafters
summary                 Prepare detailed drawings of architectural and...
sampleTitles            f more specific occupations. For additional in...
summary_tokens          [Prepare, deta

In [36]:
onet.syns[4]

[[Synset('architectural.a.01')],
 [],
 [Synset('civil.s.01'),
  Synset('civil.a.02'),
  Synset('civil.a.03'),
  Synset('civil.a.04'),
  Synset('civil.a.05'),
  Synset('civil.s.06')],
 [Synset('drafter.n.01')]]

In [37]:
### chained synonym getter passing in synsets returning flattened synonym list
def flatLemming(synList):
    synonyms = []
    for synset in synList:
        for word in synset:
            for lemma in word.lemmas():
                syn = wordnet.synsets(lemma.name())
                synonyms.append([ID.name() for ID in syn])
    return synonyms
        
def lems(ls):
    synlems = []
    for word in ls:
        synlems.append(word.lemmas())
    return synlems
        

In [38]:
#overkill
onet['flatLemmings'] = onet.syns.apply(flatLemming)
onet.flatLemmings[4][:5]

[['architectural.a.01'],
 ['civil.s.01',
  'civil.a.02',
  'civil.a.03',
  'civil.a.04',
  'civil.a.05',
  'civil.s.06'],
 ['civil.s.01',
  'civil.a.02',
  'civil.a.03',
  'civil.a.04',
  'civil.a.05',
  'civil.s.06'],
 ['polite.a.01', 'civilized.s.02', 'civil.a.02'],
 ['civil.s.01',
  'civil.a.02',
  'civil.a.03',
  'civil.a.04',
  'civil.a.05',
  'civil.s.06']]

In [44]:
onet.head()

Unnamed: 0     Code                                      Occupation_x  \
0           0  17-3021  Aerospace Engineering and Operations Technicians   
1           1  17-2011                               Aerospace Engineers   
2           2  17-2021                            Agricultural Engineers   
3           3  17-1011          Architects, Except Landscape and Naval     
4           4  17-3011                  Architectural and Civil Drafters   

                     Job Family  \
0  Architecture and Engineering   
1  Architecture and Engineering   
2  Architecture and Engineering   
3  Architecture and Engineering   
4  Architecture and Engineering   

                                   Career Cluster  \
0                                   Manufacturing   
1  Science, Technology, Engineering & Mathematics   
2           Agriculture, Food & Natural Resources   
3                     Architecture & Construction   
4                     Architecture & Construction   

                                 Career Pathway  \
0  Manufacturing Production Process Development   
1                    Engineering and Technology   
2         Power, Structural & Technical Systems   
3                       Design/Pre-Construction   
4                       Design/Pre-Construction   

                                       Occupation_y    fullCode  SCode  \
0  Aerospace Engineering and Operations Technicians  17-3021.00    0.0   
1                               Aerospace Engineers  17-2011.00    0.0   
2                            Agricultural Engineers  17-2021.00    0.0   
3            Architects, Except Landscape and Naval  17-1011.00    0.0   
4                  Architectural and Civil Drafters  17-3011.00    0.0   

                                         word_tokens  \
0  [Aerospace, Engineering, and, Operations, Tech...   
1                             [Aerospace, Engineers]   
2                          [Agricultural, Engineers]   
3        [Architects, Except, Landscape, and, Naval]   
4              [Architectural, and, Civil, Drafters]   

                         ...                          \
0                        ...                           
1                        ...                           
2                        ...                           
3                        ...                           
4                        ...                           

                                            synonyms  \
0  [[[aerospace]], [[technology, engineering], [e...   
1  [[[aerospace]], [[engineer, applied_scientist,...   
2  [[[agricultural], [agrarian, agricultural, far...   
3  [[[architect, designer]], [[demur, except], [e...   
4  [[[architectural]], [], [[civil], [civil, poli...   

                                    summary_synonyms  \
0  [[[operate, run], [function, work, operate, go...   
1  [[[perform, execute, do], [perform], [perform]...   
2  [[[use, utilize, utilise, apply, employ], [app...   
3  [[[plan, program, programme], [design, plan], ...   
4  [[[fix, prepare, set_up, ready, gear_up, set],...   

                                     sample_synonyms  \
0  [[[avionics]], [[technician], [technician]], [...   
1  [[[aeronautical, aeronautic]], [[engineer, app...   
2  [[[agricultural], [agrarian, agricultural, far...   
3  [[[architect, designer]], [[architectural]], [...   
4  [[[degree_Fahrenheit, F], [fluorine, F, atomic...   

                                        flatSynonyms  \
0  [aerospace, technology, engineering, engineeri...   
1  [aerospace, engineer, applied_scientist, techn...   
2  [agricultural, agrarian, agricultural, farming...   
3  [architect, designer, demur, except, exclude, ...   
4  [architectural, civil, civil, polite, civil, c...   

                                summary_flatSynonyms  \
0  [operate, run, function, work, operate, go, ru...   
1  [perform, execute, do, perform, perform, do, p...   
2  [use, utilize, utilise, apply, employ, apply, ...   
3  [plan, program,

In [42]:
def matcher(resume):
    list_of_positions_lists_of_scores = []
    for list_of_job_descriptions in resume:
        list_of_positions_lists_of_scores.append(onetJobCycler(list_of_job_descriptions, onet.syns))
        print(len(list_of_positions_lists_of_scores[0]))
            #word_score_list = []
    return list_of_positions_lists_of_scores
            
def onetJobCycler(jobDescription, jobPotentialsList):
    job_scores = []
    for i,job in enumerate(jobPotentialsList):
    #for i,job in enumerate(onet.syns): #all_possible_job_descriptions:
        job_score_tracker = 0
        #print(jobDescription)
        for wordsyn in jobDescription:
            #record Wu-Palmer similarity score to each word in each job category description in Onet data
            job_score_tracker += onetWordCycler(wordsyn, job)
        job_scores.append(job_score_tracker)
    return job_scores
            
#word_score_list.append(word_score_tracker) #save word scores to list for analysis if needed

def onetWordCycler(word, pos):
    word_score = 0
    for w in pos:
        for syns in w:
            #print(syns)
            if syns:
                wup_score = word.wup_similarity(syns)
                #print(wup_score)
                if wup_score:
                    word_score += wup_score
    return word_score
            
            
            
    
    

In [43]:
dfwx['potentialJob_scores'] = dfwx.syns.apply(matcher)

NameError: name 'dfwx' is not defined